In [9]:
import numpy as np
import sympy
import scipy
import torch

emc = float( sympy.S.EulerGamma.n(10) )
emc

0.5772156649036333

The update formula for Weibull alpha (scale parameter): solve for alpha in the following equation

$$
\sum_{i=1}^N [\log Y_i - \mu - \hat{z}_i^{(k)}] - e^{-K} \cdot \sum_{i=1}^N \exp(\alpha (\log Y_i - \mu - \hat{z}_i^{(k)}) + \alpha^2 / 2 \xi^{(k)}) \cdot ( (\log Y_i - \mu - \hat{z}_i^{(k)}) + \alpha / \xi^{(k)} ) = 0 \text{ where } K \text{ is an Euler-Mascheroni constant}( \approx 0.57722) \\
$$

In the following code block and all the others that follow, the variable "emc" holds the Euler Mascheroni constant. This constant is denoted by K in the equation above as well as the other equations in this notebook. <br>
In the code below z_hat, y are vectors of length N and the other parameters are scalars. Also note that the superscript (lowercase k) can be disregarded (it denotes the iteration of number of VAMP and it is different from the Euler Mascheroni constant capital K).

In [7]:
def update_Weibull_alpha_eq(alpha, y, mu, z_hat, xi):
    n,_ = y.shape
    out = np.zeros(n)
    res = np.log(y) - mu - z_hat
    out = np.sum(res) - np.exp(-emc) * np.sum( np.exp(alpha * res + (alpha**2)/2/xi) * (res + alpha/xi) )
    return out

def update_Weibull_alpha(y, mu, z_hat, alpha_old, xi):
    alpha_new = scipy.optimize.fsolve(update_Weibull_alpha_eq, x0 = alpha_old, args=(y, mu, z_hat, xi))
    # This is necessary because scipy fsolve provides its roots in a list
    if isinstance(alpha_new, np.ndarray) or isinstance(alpha_new, list): alpha_new = float(alpha_new[0])
    return alpha_new

The update formula for Weibull mu

$$
\mu = \frac{-K}{\alpha} + \frac{\alpha}{2 \xi^{(k)}} + \frac{1}{\alpha} \log \left( \sum_{i=1}^{n} \exp \left( \alpha (\log Y_i - \hat{z}_i^{(k)}) \right) \right) - \frac{1}{\alpha} \log N
$$

In [2]:
def update_Weibull_mu(y, z_hat, alpha, xi):
    n,_ = y.shape
    mu_new = - np.log(n) / alpha - emc/alpha + alpha / 2 / xi + 1 / alpha * np.log(np.sum(np.exp(alpha*(np.log(y) - z_hat))))
    if isinstance(mu_new, np.ndarray) or isinstance(mu_new, list): mu_new = float(mu_new[0])
    return mu_new

Weibull likelihood

$$
\sum_{1}^{N} ( \text{    } -\mathrm{e}^{-a{\mu} - az_{hat} - K} \left(Y^{a} \mathrm{e}^{\frac{a^{2}}{2{\xi}}} + \left(a\mathrm{e}^{az_{hat} + K} {\mu} + \left(\mathrm{e}^{K} az_{hat} - \mathrm{e}^{K} \ln\left(Y\right) \, a + K\mathrm{e}^{K}\right) \mathrm{e}^{az_{hat}}\right) \mathrm{e}^{a{\mu}}\right) \text{    })
$$

In [4]:
def weibull_likelihood(z_hat, Y, a, mu, xi, K):
    term1 = torch.exp(-a * mu - a * z_hat - K)
    term2 = Y ** a * torch.exp(a ** 2 / (2 * xi))
    term3 = a * torch.exp(a * z_hat + K) * mu
    term4 = (torch.exp(K) * a * z_hat - torch.exp(K) * torch.log(Y) * a + K * torch.exp(K)) * torch.exp(a * z_hat)
    term5 = torch.exp(a * mu)

    result = -term1 * (term2 + (term3 + term4) * term5)
    return torch.sum(result)